# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [2]:
import pandas as pd
df=pd.read_csv('/datasets/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

есть пропуски в столбцах: days_employed, total_incomе, так же не правильные типы данных в столбцах days_employed и total_income

## Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
days_employed_mean=df['days_employed'].mean()
df['days_employed'] = df['days_employed'].abs()

medians=(df.groupby(['income_type','education']).agg({'total_income':'median'}).rename(columns={'total_income':'med_total_income'}))
df=df.merge(medians, on = ['income_type','education'])
df[['income_type','education','total_income','med_total_income']][df['total_income'].isna()]
df.loc[df['total_income'].isna(),'total_income']=df.loc[df['total_income'].isna(),'med_total_income']

df['days_employed'] = df['days_employed'].fillna(days_employed_mean)


**Вывод**

в столбцах days_employed и total_income пропущенные значения, причём они находятся в одних и тех же строках.
так же в этих столбцах есть отрицательные значения. для заполнения пропусков необходимо вписать в пустые ячейки среднее значения по столбцам и перевести отрицательные числа в положительные. затем пишем функцию которая проверяет числа и если оно отрицательное то функция ее умножает на -1 и возвращает число. 


### Замена типа данных

In [3]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
med_total_income    21525 non-null float64
dtypes: float64(1), int64(7), object(5)
memory usage: 2.3+ MB


**Вывод**

перевели тип данных столбцов с помощью метода astype из вещественного в целочисленное

### Обработка дубликатов

In [4]:
df.drop_duplicates()
from collections import Counter
Counter(df['education'])
df['education'] = df['education'].str.lower()
Counter(df['education'])
Counter(df['family_status'])
Counter(df['income_type'])
Counter(df['purpose'])
Counter(df['children'])
df['children'] = df['children'].replace(-1, 1)
children_median = df.loc[df.loc[:, 'children'] != 20]['children'].median()
df['children'] = df['children'].replace(20, children_median)
Counter(df['children'])

Counter({1: 4865, 0: 14225, 2: 2055, 4: 41, 3: 330, 5: 9})

**Вывод**

проверяем каждый столбец на ошибки, с помощью контейнера. находим ошибки в столбце с образованием, так как в нем есть значения с разным регистром, с помощью str.lower() приводим к одному значению и тем самым уюираем дубликаты. Далее в столбце children находим что написаны не верные количества детей это -1 и 20. скорее всего ошибка произошла из-за невнимательного ввода данных. Данные ошибки исправляем при помощи метода replace и вместо 20 ставим медианное значение. находим медиану при помощи функции loc и далее с помощью replace меняем на медианное значение. 

### Лемматизация

In [5]:
from pymystem3 import Mystem
m = Mystem()
text='|'.join(df['purpose'])
lemmas = m.lemmatize(text)
from collections import Counter
Counter(lemmas)


Counter({'покупка': 5912,
         ' ': 33677,
         'жилье': 4473,
         '|': 21524,
         'строительство': 1881,
         'недвижимость': 6367,
         'сыграть': 774,
         'свадьба': 2348,
         'собственный': 635,
         'образование': 4022,
         'автомобиль': 4315,
         'на': 2233,
         'подержать': 836,
         'свой': 2235,
         'операция': 2610,
         'с': 2924,
         'профильный': 436,
         'сделка': 944,
         'со': 630,
         'коммерческий': 1315,
         'приобретение': 462,
         'высокий': 1375,
         'получение': 1316,
         'подержанный': 132,
         'жилой': 1233,
         'для': 1294,
         'сдача': 653,
         'заниматься': 908,
         'проведение': 777,
         'дополнительный': 909,
         'ремонт': 612,
         'семья': 641,
         '\n': 1})

**Вывод**

разбили стоблец на леммы пр помощи join. он позваляет строки столбца объединять в одну строчку присоеденяя их к друг другу через заданный символ. в данном случае это |. разбив на леммы данны столбца м ы можем увидеть что есчть много одинаковых целей, но написанны они по разному. в категоризации необходимо сделать так, чтобы данные цель с разным написанием но одинаковым по смыслу записывались в одну категорию. 

### Категоризация данных

In [6]:

def cat(row):
    purpose = row['purpose']
    lemmas = m.lemmatize(purpose)
    
    if 'автомобиль' in lemmas: return 'автомобиль'
    if 'свадьба' in lemmas: return 'свадьба'
    if 'образование' in lemmas: return 'образование'
    if 'недвижимость' in lemmas: return 'недвижимость'
    if 'жилье' in lemmas: return 'недвижимость'
    if 'жилой' in lemmas: return 'недвижимость'
    return 'прочие'
df['purpose_group'] = df.apply(cat, axis = 1)
print(df['purpose_group'].value_counts())

недвижимость    10840
автомобиль       4315
образование      4022
свадьба          2348
Name: purpose_group, dtype: int64


**Вывод**

далее нам необходимо проверить слова которые мы выбрали на их наличие в в столбце. создаем функцию которая разбив столбец на слова начинает проверять наличие заданных слов в этом столбце. когда данное условие выполнено то сумму данных слов записываем в новый столбец при помощи apply. проведя данную категоризацию можем увидеть что почти 50% запшенных кредитов это кредиты на недвижимость на втором месте автомобиль и на третьем это образование. 

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
debt_from_children = pd.DataFrame()
debt_from_children['count_debt'] = df.groupby('children')['debt'].count()
debt_from_children['sum_debt'] = df.groupby('children')['debt'].sum()
debt_from_children['result %'] = debt_from_children['sum_debt'] / debt_from_children['count_debt'] *100
debt_from_children.sort_values('result %', ascending = False)

,count_debt,sum_debt,result %
children,,,
4,41,4,9.756098
2,2055,194,9.440389
1,4865,445,9.146968
3,330,27,8.181818
0,14225,1071,7.528998
5,9,0,0.000000


**Вывод**

при помощи группировки по столбцу children создаем таблицу где создаем новые столбцы count_debt(количество взятых кредитов с данным количеством детей), sum_debt (количество просрочек в кредите с данным. количеством детей) и result (отношение просрочек к взятым кредитам). из построения данной таблицы можно выяснить, чем больше детей тем выше уровень просрочки  

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
debt_from_family_status = pd.DataFrame()

debt_from_family_status['count_debt'] = df.groupby('family_status')['debt'].count()
debt_from_family_status['sum_debt'] = df.groupby('family_status')['debt'].sum()
debt_from_family_status['result %'] = debt_from_family_status['sum_debt'] / debt_from_family_status['count_debt']*100 
debt_from_family_status.sort_values('result %', ascending = False)

,count_debt,sum_debt,result %
family_status,,,
Не женат / не замужем,2813,274,9.740491
гражданский брак,4177,388,9.288963
женат / замужем,12380,931,7.520194
в разводе,1195,85,7.112971
вдовец / вдова,960,63,6.562500


**Вывод**

при помощи группировки по столбцу family_status создаем таблицу где создаем новые столбцы count_debt(количество взятых кредитов с по семейному пложению), sum_debt (количество просрочек в кредите с данным поо семейному) и result (отношение просрочек к взятым кредитам). из построения данной таблицы можно увидеть, что больше всего просрочек по кредиту у людей имеющие статус "Не женат / не замужем", а меньше всего имеют просрочки люди со статусом "в разводе" и "вдовец"

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
def income_category(value):
    if value >= 500000:
        return 10
    else:
        value = value // 50000
        return value
df['total_income_id'] = df['total_income'].apply(income_category)
debt_income = pd.DataFrame()
debt_income['count_debt'] = df.groupby('total_income_id')['debt'].count()
debt_income['sum_debt'] = df.groupby('total_income_id')['debt'].sum()
debt_income['result %'] = debt_income['sum_debt'] / debt_income['count_debt']*100
debt_income.sort_values('result %', ascending = False)


,count_debt,sum_debt,result %
total_income_id,,,
2,7075,617,8.720848
3,4734,404,8.534009
6,624,51,8.173077
1,4093,331,8.086978
7,330,24,7.272727
4,2437,172,7.057858
5,1330,88,6.616541
8,197,13,6.598985
10,222,14,6.306306


**Вывод**

ттак как чисел с уровнем дохода очень много. необходимо разделить людей на группы по уровню дохода и это будут группы с шагом 50000 рублей. 0 гр.- менеее 50000 рублей, 1 гр - до 50000 и так далее. для этого напишем функцию где будем проверять значения. если зарплата у человека больше или равна 500 т., то его сразу записываем в группу 10. далее будем делить доход на 50000 и делаим по группам остальных людей. и согласно нашей таблице можно увидеть что имеющие заработок от 350 т. руб имеют меньше просрочек чем с люди с заработком ниже  так же маленький процент просрока у людей с зп до 50000

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
debt_from_purpose = pd.DataFrame()

debt_from_purpose['count_debt'] = df.groupby('purpose_group')['debt'].count()
debt_from_purpose['sum_debt'] = df.groupby('purpose_group')['debt'].sum()
debt_from_purpose['result %'] = debt_from_purpose['sum_debt'] / debt_from_purpose['count_debt']*100
debt_from_purpose.sort_values('result %', ascending = False)

,count_debt,sum_debt,result %
purpose_group,,,
автомобиль,4315,403,9.339513
образование,4022,370,9.199403
свадьба,2348,186,7.921635
недвижимость,10840,782,7.214022


**Вывод**

по аналогии с верхними задачами группируем и вычисляем. видим что чаще всего просроков в платежах у людей которые брали кредиты на автомобиль и образование. а остальные платят более менее в срок

## Шаг 4. Общий вывод

Количество детей влияет на платежи в срок. 
Семейный статус влияет так же влияет на платежи в срок. самые не платежеспосбоные люди не состоящие в браке.
Уровень дохода не сильно влияет на платежи. 
Чаще всего в просрок идут платежи по кредитам которые взяли на покупку авто или на образование. 

Можно нарисовать картинку самого неплатежеспособного клиента селовек котрый в статусе "не замужем/не женат" с уровнем дохода около 150т.рублей, коотрый хочет взять кредит на автомобиль. 

Общий вывод - семейный статус влияет на вероятность платежей по кредиту в срок - люди, бывший в браке чаще плятят в срок чем те, кто не бывали в браке, причем разведенные и овдовевшие платят в срок чаще, чем люди в браке. Чем больше детей, тем чаще платят не в срок, бездетные реже просрачивают оплату.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [х]  файл изучен;
- [х]  определены пропущенные значения;
- [х]  заполнены пропущенные значения;
- [х]  есть пояснение, какие пропущенные значения обнаружены;
- [х]  описаны возможные причины появления пропусков в данных;
- [х]  объяснено, по какому принципу заполнены пропуски;
- [х]  заменен вещественный тип данных на целочисленный;
- [х]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [х]  удалены дубликаты;
- [х]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [х]  описаны возможные причины появления дубликатов в данных;
- [х]  выделены леммы в значениях столбца с целями получения кредита;
- [х]  описан процесс лемматизации;
- [х]  данные категоризированы;
- [х]  есть объяснение принципа категоризации данных;
- [х]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [х]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [х]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [х]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [х]  в каждом этапе есть выводы;
- [х]  есть общий вывод.